In [1]:
import nltk
from nltk import trigrams
from nltk.corpus import brown
from collections import defaultdict

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
uniModel = defaultdict(lambda: 0)
biModel = defaultdict(lambda: defaultdict(lambda: 0))
triModel = defaultdict(lambda: defaultdict(lambda: 0))
model1 = defaultdict(lambda: set())
model2 = defaultdict(lambda: set())

In [4]:
def convert_to_lower(c):
  if type(c)==str:
    return c.lower()
  return c

In [5]:
totalWord = 0
for sentence in brown.sents():
  for word in sentence:
    uniModel[word] += 1
    totalWord +=1

  for firstWord, secondWord, thirdWord in trigrams(sentence, pad_right=True, pad_left=True):
    firstWord = convert_to_lower(firstWord)
    secondWord = convert_to_lower(secondWord)
    thirdWord = convert_to_lower(thirdWord)

    biModel[thirdWord][secondWord] += 1
    triModel[thirdWord][firstWord] += 1

    model1[secondWord].add(thirdWord)
    model2[firstWord].add(thirdWord)

In [6]:
def calculate_probablities(model):
  for nextWord in model:
    totalCount = float(sum(model[nextWord].values()))
    for prevWord in model[nextWord]:
        model[nextWord][prevWord] /= totalCount

calculate_probablities(biModel)
calculate_probablities(triModel)

for word in uniModel:
  uniModel[word] /= totalWord

In [7]:
def calculate_trigram_Probability(firstWord, secondWord, thirdWord):
  return uniModel[thirdWord]*biModel[thirdWord][secondWord]*triModel[thirdWord][firstWord]

In [8]:
def predict_word(firstWord,secondWord):
  response = []
  for thirdWord in model1[secondWord] & model2[firstWord]:
    res = calculate_trigram_Probability(firstWord,secondWord,thirdWord)
    response.append((thirdWord,res))
  return response

In [9]:
while(True):
    text = input("Enter a line (In lower charecter): ")
    if text == "quit":
        print("#############################################")
        print("######           Terminated          ########")
        print("#############################################")
        break
    
    else:
        try:
            text = text.split(" ")
            data = predict_word(text[-2], text[-1])
            sorted(data, key=lambda o: o[1], reverse=True)
            print("################## SUGGESTIONS ##################")
            print(data[:10])
            print()
            print()
            
        except:
            continue

Enter a line (In lower charecter): i love my
################## SUGGESTIONS ##################
[('memory', 4.472908715504856e-08), ('comrades', 8.611840246918684e-08), ('life', 2.3913870437724607e-08), ('god', 5.8644962424938966e-09), ('country', 2.5595318520791697e-08), (',', 7.086233274798519e-10), ('visit', 1.5801541737465474e-08), ('only', 1.3917594982459344e-09), ('!', 1.0791779758043463e-09), ('woman', 3.7244286782153897e-09)]


Enter a line (In lower charecter): how to
################## SUGGESTIONS ##################
[('mother', 3.5255090174071254e-09), ('speak', 4.1123316484872855e-07), ('take', 5.581981461417172e-07), ('your', 7.194922832943639e-08), ('merely', 6.284635132182085e-09), ('death', 8.592872177456951e-08), ('other', 1.4994296698493845e-08), ('american', 0.0), ('work', 2.21716576510198e-07), ('with', 3.409755405741027e-10)]


Enter a line (In lower charecter): i am having
################## SUGGESTIONS ##################
[('and', 5.775394317851232e-11), ('light', 2